# Making a function for the decoding
Testing on a small dataset


#### Code used for batches below 

In [ ]:
# !pip install xdot --user

In [1]:
import numpy as np
import nilearn
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.pipeline import Pipeline

from beaker.cache import CacheManager
from beaker.util import parse_cache_config_options
import objgraph
# import xdot
import pdb

import multiprocessing
from multiprocessing import Pool

import time
from functools import partial

/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec_

## Cache Money

In [2]:
cache_opts = {
    'cache.type': 'file',
    'cache.data_dir': '/Users/gracer/Google Drive/decode/tmp/data',
    'cache.lock_dir': '/Users/gracer/Google Drive/decode/tmp/lock',
    'cache.regions' : 'short_term, long_term'
}

cache = CacheManager(**parse_cache_config_options(cache_opts))

tmpl_cache = cache.get_cache('/Users/gracer/Google Drive/decode/tmp/mytemplate.html', type='dbm', expire=3600)

will want to cache the following:
* masker
* the fit (X)
* anova_svc
* y_pred


## Fitter function 

In [22]:
@cache.cache('short_term')
def fitter(IMAG_MASK, DATASET, LABELS ,CONDITION_MASK, ANOVA_SVC, CONDITIONS):
    masker = NiftiMasker(mask_img=IMAG_MASK, standardize=True, memory="nilearn_cache", memory_level=1)
    X = masker.fit_transform(DATASET.func[0])
    print(X.shape)
    pdb.set_trace()
    # Apply our condition_mask
    X = X[CONDITION_MASK]
    
    return([ANOVA_SVC.fit(X,CONDITIONS),X])


In [23]:
# NESTED CROSS VALIDATION

def run_CV(MODEL,X,y ,PARAMS):
    from sklearn.model_selection import GridSearchCV
    grid = GridSearchCV(MODEL, param_grid={'anova__k': PARAMS}, verbose=1, cv=5, n_jobs=4)
    return cross_val_score(grid, X, y, cv=5)
        


In [28]:
def main(DATASET, MASK, BEHAVIORAL):
    from sklearn.svm import SVC
    from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
    from sklearn.pipeline import Pipeline

    svc = SVC(kernel='linear')
    FEATURE_SELECTION = SelectKBest(f_classif, k=500)
    ANOVA_SVC = Pipeline([('anova', FEATURE_SELECTION), ('svc', svc)])
    
    #image mask
    IMAG_MASK=MASK

    BEHAVIORAL = BEHAVIORAL
    print(BEHAVIORAL['labels'].shape)
   
    CONDITIONS = BEHAVIORAL['labels']

    #our dataset concatenated image 
    DATASET=DATASET

    LABELS = BEHAVIORAL["labels"]
    print(LABELS.shape)
    #restrict data to our target analysis 
    CONDITION_MASK = BEHAVIORAL['labels'].isin(['face', 'house'])
    CONDITIONS = CONDITIONS[CONDITION_MASK]

    #confirm we have the # of condtions needed
    print(CONDITIONS.unique())
    
    k_range = [[ 15, 50, 150, 300], [500, 1000, 3000, 5000]]
    
    [ANOVA_SVC_FIT,X]=fitter(IMAG_MASK, DATASET, LABELS ,CONDITION_MASK, ANOVA_SVC, CONDITIONS)
    NUM=run_CV(ANOVA_SVC_FIT, X, CONDITION_MASK, k_range)
    return(ANOVA_SVC_FIT)
    
    
    
    

In [29]:
main(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" "))

(1452,)
(1452,)
['face' 'house']


ValueError: Found input variables with inconsistent numbers of samples: [216, 1452]

In [ ]:

def parallel_runs(data_list):
    pool = multiprocessing.Pool(processes=4)
    ANOVA_SVC_FIT = test(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" "))
    pdb.set_trace()
    
    prod_x=partial(run_CV, MODEL=ANOVA_SVC_FIT) 
    result_list = pool.map(prod_x, data_list) 
    print(result_list)



In [ ]:
if __name__ == '__main__':
    start = time.time()
    parallel_runs(k_range)
    print("Nested CV score: %.4f" % np.mean(nested_cv_scores))
    print("Time: ", (time.time() - start))